In [0]:
df = spark.read.format("csv").option("header", "true").load("/Volumes/databricks_telco_customer_dataset/v01/telco/telco-customer-churn.csv")
display(df)

In [0]:
import os
import joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
)

# ---------- 1. Load data ----------
DATA_PATH = "/Volumes/databricks_telco_customer_dataset/v01/telco/telco-customer-churn.csv"
df = pd.read_csv(DATA_PATH)

# ---------- 2. Basic cleaning ----------

# Convert TotalCharges to numeric (it comes as string with some blanks)
if "TotalCharges" in df.columns:
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
    # Drop rows where TotalCharges can't be converted (usually a tiny number)
    df = df.dropna(subset=["TotalCharges"])

# Drop customerID – it's just an identifier
if "customerID" in df.columns:
    df = df.drop(columns=["customerID"])

# Target column
TARGET_COL = "Churn"
y = df[TARGET_COL].map({"Yes": 1, "No": 0})  # binary 0/1
X = df.drop(columns=[TARGET_COL])

# ---------- 3. Feature types ----------

# Infer numeric and categorical columns
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object", "bool"]).columns.tolist()

print("Numeric features:", numeric_features)
print("Categorical features:", categorical_features)

# ---------- 4. Preprocessing + model pipeline ----------

numeric_transformer = "passthrough"

categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

clf = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",  # churn is usually imbalanced
    n_jobs=-1,
)

model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("clf", clf),
    ]
)

# ---------- 5. Train / test split ----------

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

# ---------- 6. Train ----------
print("Training model...")
model.fit(X_train, y_train)

# ---------- 7. Evaluate ----------
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_proba)
cm = confusion_matrix(y_test, y_pred)

print(f"\nAccuracy: {acc:.4f}")
print(f"ROC-AUC:  {roc:.4f}\n")

print("Classification report:")
print(classification_report(y_test, y_pred, target_names=["No churn", "Churn"]))

print("Confusion matrix:")
print(cm)

# ---------- 8. Save model ----------
os.makedirs("artifacts", exist_ok=True)
MODEL_PATH = os.path.join("artifacts", "telco_churn_model.joblib")
joblib.dump(model, MODEL_PATH)
print(f"\nModel saved to: {MODEL_PATH}")


In [0]:
import joblib
import pandas as pd

model = joblib.load("artifacts/telco_churn_model.joblib")

sample = pd.DataFrame(
    [
        {
            "gender": "Female",
            "SeniorCitizen": 0,
            "Partner": "Yes",
            "Dependents": "No",
            "tenure": 5,
            "PhoneService": "Yes",
            "MultipleLines": "No",
            "InternetService": "Fiber optic",
            "OnlineSecurity": "No",
            "OnlineBackup": "Yes",
            "DeviceProtection": "No",
            "TechSupport": "No",
            "StreamingTV": "Yes",
            "StreamingMovies": "Yes",
            "Contract": "Month-to-month",
            "PaperlessBilling": "Yes",
            "PaymentMethod": "Electronic check",
            "MonthlyCharges": 80.5,
            "TotalCharges": 400.0,
        }
    ]
)

proba = model.predict_proba(sample)[:, 1][0]
pred = model.predict(sample)[0]

print(f"Churn probability: {proba:.3f}")
print("Predicted label:", "Churn" if pred == 1 else "No churn")
